In [ ]:
import pandas as pd;
import numpy as np;
import folium;
from folium.plugins import MarkerCluster, HeatMap;
import webbrowser;

desired_columns=['Name_of_Sc', 'Level_', 'Status', 'SchSponsor', 'Type1', 'Type2','Type3', 'PupilTeach', 'ClassrmRat','TotalEnrol','Province', 'District', 'Division','Location', 'Costituenc', 'Latitude', 'Longitude', 'Total Teachers'];
df=pd.read_csv("C:\\Users/HomePC/Downloads/kenya_primary_schools_updated.csv");
df_clean=df[desired_columns];


df_clean=df_clean[df_clean['Latitude'].between(-4.7,4.6) & df_clean['Longitude'].between(33.9,41.9)]

df_clean['Required_Teachers']=df_clean['TotalEnrol']/40
df_clean['Teacher_Gap']=df_clean['Required_Teachers']-df_clean['Total Teachers']
df_clean['Teacher_Gap']=df_clean['Teacher_Gap'].apply(lambda x: max(0,x))
#Teacher shortage category
conditions = [
    df_clean['PupilTeach']==0,
    df_clean['PupilTeach']<=2,
    df_clean['PupilTeach']<=5,
    df_clean['PupilTeach']>5
]
choices=['No shortage','Mild(1-2)','Moderate(3-5)','Severe(5+)']
df_clean['Shortage_Level']=np.select(conditions, choices, default='Unknown')

constituency_gap=df_clean.groupby('Costituenc').agg(
    total_students=('TotalEnrol','sum'),
    total_teachers=('Total Teachers','sum'),
    total_teacher_gap=('Teacher_Gap','sum'),
    school_count=('Name_of_Sc', 'count')
).reset_index()
constituency_gap['teachers_needed'] = constituency_gap['total_students'] / 40
constituency_gap['total_teacher_gap'] = constituency_gap['teachers_needed'] - constituency_gap['total_teachers']

map_kenya=folium.Map(location=[-0.0236,37.9062], zoom_start=7, tiles='OpenStreetMap')
folium.TileLayer(
    tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    attr='OpenStreetMap',
    name='OpenStreetMap'
).add_to(map_kenya)

color_map={
    'No shortage': 'green',
    'Mild(1-2)': 'blue',
    'Moderate(3-5)': 'orange',
    'Severe(5+)': 'red'
}


#Shortage heatmap
heat_data = [[row['Latitude'], row['Longitude'], row['Teacher_Gap']]
             for _, row in df_clean[df_clean['Teacher_Gap']>0].iterrows()]
HeatMap(heat_data, name='Teacher Shortage Intensity', radius=15, gradient={0.4: 'blue', 0.65: 'lime', 1:'red'}).add_to(map_kenya)

#Layer Control
folium.LayerControl().add_to(map_kenya)

map_kenya.save('Kenya_Teacher_Gap_Map.html')
webbrowser.open('Kenya_Teacher_Gap_Map.html')


ModuleNotFoundError: No module named 'folium'